In [ ]:
# SPLITING DATA FROM DELIMITED TEXT FILE
# AND STORING IT IN DATABASE

In [2]:
#CONNECTION TO THE DATABASE
import mysql.connector as mysql
from mysql.connector import Error

try:
    instance = mysql.connect(host='localhost',database='incubyte',user='root',password='mysql@123')
    if instance.is_connected():
        db_info=instance.get_server_info()
        print("Connected...",db_info)
except Error as e:
    print("Error occured...",e)

Connected... 8.0.23


In [3]:
# mycursor=instance.cursor(buffered=True)
# # SPLITING DATA FROM DELIMITED FILE

# data=(name, cid,opendate,closedate,vacctype,dr,state,country,pc,activeCust)

# # def primaryCheck(name,cp):
# #     print(cp)
# #     checkQuery = "select CustomerName from customer where CustomerName='%s'"%name
# #     mycursor.execute(checkQuery)
# #     output =mycursor.fetchall()
# #     for j in output:
# #             if (j[0]==name):
# #                 print(name," is already inserted to the Customer table.")
# #                 cp=1
# #                 return cp
# #             else:
# #                 cp=0
# #                 return cp
            
# with open('DelimitedData.txt') as f:
#     lines = f.readlines()
#     for line in lines:        
#         name=line.split('|')[2]
#         cid=line.split('|')[3]
#         opendate=line.split('|')[4]
#         closedate=line.split('|')[5]
#         vacctype=line.split('|')[6]
#         dr=line.split('|')[7]
#         state=line.split('|')[8]
#         country=line.split('|')[9]
#         pc=0
#         dob=line.split('|')[10]
#         activeCust=line.split('|')[11]

#         print(name,"\n",cid,"\n","OpenDate: ",opendate[0:4],"/",opendate[4:6],"/",opendate[6:8],"\n","CloseDate: ",closedate[0:4],"/",closedate[4:6],"/",closedate[6:8],"\n",vacctype,"\n",dr,"\n",state,"\n",country,"\n",dob,"\n",activeCust)
        
#         cp=0
#         primaryCheck(name,cp)
#         print(cp)
        
# #       INSERT QUERY IN CUSTOMER TABLE
        
#         instQuery = """INSERT INTO customer(CustomerName,CustomerID ,CustomerOpenDate ,LastConsultedDate,VaccinationType ,DoctorConsulted ,State ,Country,PostCode,ActiveCustomer) 
#         VALUES (%s, %s, %s, %s, %s,%s,%s,%s,%s,%s)"""
#         mycursor.execute(instQuery,data)

# #       CHECKING IF COUNTRY TABLE EXISITS OR NOT!  
#         flag=0
#         showQuery = "show tables;"
#         mycursor.execute(showQuery)
#         result =mycursor.fetchall()
#         for i in result:
#             if (i[0].upper()==country):
#                 flag=1
#                 break

#         if (flag==0):
# #           IF TABLE DOES NOT EXISITS THEN IT WILL CREATE A NEW TABLE OF COUNTRY NAME AND INSERT DATA IN IT!  
#             mycursor.execute("""CREATE TABLE """ +country+ """ (
#               `CustomerName` VARCHAR(255) NOT NULL,
#               `CustomerID` VARCHAR(18) NOT NULL,
#               `CustomerOpenDate` DATE NOT NULL,
#               `LastConsultedDate` DATE NULL,
#               `VaccinationType` CHAR(5) NULL,
#               `DoctorConsulted` CHAR(255) NULL,
#               `State` CHAR(5) NULL,
#               `Country` CHAR(5) NULL,
#               `PostCode` INT(5) NULL,
#               `DOB` DATE NULL,
#               `ActiveCustomer` CHAR(1) NULL,
#               PRIMARY KEY (`CustomerName`))""")

#             newsql = """INSERT INTO """+ country+"""(CustomerName,CustomerID ,CustomerOpenDate ,LastConsultedDate,VaccinationType ,DoctorConsulted ,State ,Country,PostCode,ActiveCustomer) 
#             VALUES (%s, %s, %s, %s, %s,%s,%s,%s,%s,%s)"""
#             mycursor.execute(newsql,data)
#         else:
# #           IF TABLE EXISITS THEN IT WILL INSERT DATA IN IT!              
#             wellsql = """INSERT INTO """+ country+"""(CustomerName,CustomerID ,CustomerOpenDate ,LastConsultedDate,VaccinationType ,DoctorConsulted ,State ,Country,PostCode,ActiveCustomer) 
#             VALUES (%s, %s, %s, %s, %s,%s,%s,%s,%s,%s)"""
#             mycursor.execute(wellsql,data)
        
        
# print("------------------------------------------------")

# instance.commit()  
# mycursor.close()


In [27]:
mycursor=instance.cursor(buffered=True)

# SPLITING DATA FROM DELIMITED FILE

try: 
    with open('DelimitedData.txt') as f:
        lines = f.readlines()
        for line in lines:       
            if (line.split('|')[1]=='D'):
                name=line.split('|')[2]
                cid=line.split('|')[3]
                opendate=line.split('|')[4]
                closedate=line.split('|')[5]
                vacctype=line.split('|')[6]
                dr=line.split('|')[7]
                state=line.split('|')[8]
                country=line.split('|')[9]
                pc=0
                dob=line.split('|')[10][4:8]+"/"+line.split('|')[10][2:4]+"/"+line.split('|')[10][0:2]
                activeCust=line.split('|')[11]

                print(name,"\n",dob,"\n",cid,"\n","OpenDate: ",opendate[0:4],"/",opendate[4:6],"/",opendate[6:8],"\n","CloseDate: ",closedate[0:4],"/",closedate[4:6],"/",closedate[6:8],"\n",vacctype,"\n",dr,"\n",state,"\n",country,"\n",dob,"\n",activeCust)
                try:
                    
            #       INSERT QUERY IN CUSTOMER TABLE
                    data=(name, cid,opendate,closedate,vacctype,dr,state,country,pc,dob,activeCust)
                    insql = """INSERT INTO customer(CustomerName,CustomerID ,CustomerOpenDate ,LastConsultedDate,VaccinationType ,DoctorConsulted ,State ,Country,PostCode,DOB,ActiveCustomer) 
                    VALUES (%s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s)"""
                    mycursor.execute(insql,data)

            #       CHECKING IF COUNTRY TABLE EXISITS OR NOT!  
                    flag=0
                    showQuery = "show tables;"
                    mycursor.execute(showQuery)
                    result =mycursor.fetchall()
                    for i in result:
                        if (i[0].upper()==country):
                            flag=1
                            break

                    if (flag==0):
            #           IF TABLE DOES NOT EXISITS THEN IT WILL CREATE A NEW TABLE OF COUNTRY NAME AND INSERT DATA IN IT!  
                        mycursor.execute("""CREATE TABLE """ +country+ """ (`CustomerName` VARCHAR(255) NOT NULL,
                          `CustomerID` VARCHAR(18) NOT NULL,
                          `CustomerOpenDate` DATE NOT NULL,
                          `LastConsultedDate` DATE NULL,
                          `VaccinationType` CHAR(5) NULL,
                          `DoctorConsulted` CHAR(255) NULL,
                          `State` CHAR(5) NULL,
                          `Country` CHAR(5) NULL,
                          `PostCode` INT(5) NULL,
                          `DOB` DATE NULL,
                          `ActiveCustomer` CHAR(1) NULL,
                          PRIMARY KEY (`CustomerName`))""")

                        newsql = """INSERT INTO """+ country+"""(CustomerName,CustomerID ,CustomerOpenDate ,LastConsultedDate,VaccinationType ,DoctorConsulted ,State ,Country,PostCode,DOB,ActiveCustomer) 
                        VALUES (%s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s)"""
                        mycursor.execute(newsql,data)
                    else:
            #           IF TABLE EXISITS THEN IT WILL INSERT DATA IN IT!              
                        wellsql = """INSERT INTO """+ country+"""(CustomerName,CustomerID ,CustomerOpenDate ,LastConsultedDate,VaccinationType ,DoctorConsulted ,State ,Country,PostCode,DOB,ActiveCustomer) 
                        VALUES (%s, %s, %s, %s, %s,%s,%s,%s,%s,%s,%s)"""
                        mycursor.execute(wellsql,data)                    
                except mysql.IntegrityError  as e:
                    print(name," is already inserted in the Customer table")
                except mysql.DataError  as e:
                    print("Data is not formatted as given in the database")
                    

except IndexError as e:
    print("Your file is not formatted properly")
except FileNotFoundError as e:
    print("Your file is not available on the location")
    

instance.commit()  
mycursor.close()


Alex 
 1987/03/06 
 123457 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 SA 
 USA 
 1987/03/06 
 A

Alex  is already inserted in the Customer table
John 
 1987/03/06 
 123458 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 TN 
 IND 
 1987/03/06 
 A

John  is already inserted in the Customer table
Mathew 
 1987/03/06 
 123459 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 WAS 
 PHIL 
 1987/03/06 
 A

Mathew  is already inserted in the Customer table
Matt 
 1987/03/06 
 12345 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 BOS 
 NYC 
 1987/03/06 
 A

Matt  is already inserted in the Customer table
Jacob 
 1987/03/06 
 1256 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 VIC 
 AU 
 1987/03/06 
 A
Jacob  is already inserted in the Customer table


True

In [8]:
# DUMMY CODE FOR TRIALS AND TESTING
with open('DelimitedData.txt') as f:
    lines = f.readlines()
#     print (lines)
    for line in lines:
        name=line.split('|')[2]
        cid=line.split('|')[3]
        opendate=line.split('|')[4]
        closedate=line.split('|')[5]
        vacctype=line.split('|')[6]
        dr=line.split('|')[7]
        state=line.split('|')[8]
        country=line.split('|')[9]
        pc=0
        dob=line.split('|')[10][4:8]+"/"+line.split('|')[10][2:4]+"/"+line.split('|')[10][0:2]
        print("here ",dob)
        activeCust=line.split('|')[11]
        
        print(name,"\n",cid,"\n","OpenDate: ",opendate[0:4],"/",opendate[4:6],"/",opendate[6:8],"\n","CloseDate: ",closedate[0:4],"/",closedate[4:6],"/",closedate[6:8],"\n",vacctype,"\n",dr,"\n",state,"\n",country,"\n",dob[4:8],"/",dob[2:4],"/",dob[0:2],"\n",activeCust)
        
# print(lines)
# f = open('well2.txt', 'w')
# for us in txt2:
#     f.write(us)

# txt1 = open("well.txt").read()
# txt2 = txt1.split("|")
# # print(txt2)
# for i in txt2:
#     if(i=="D"):
#         print(i)
    

here  1987/03/06
Alex 
 123457 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 SA 
 USA 
 /03/ / 87 / 19 
 A

here  1987/03/06
John 
 123458 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 TN 
 IND 
 /03/ / 87 / 19 
 A

here  1987/03/06
Mathew 
 123459 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 WAS 
 PHIL 
 /03/ / 87 / 19 
 A

here  1987/03/06
Matt 
 12345 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 BOS 
 NYC 
 /03/ / 87 / 19 
 A

here  1987/03/06
Jacob 
 1256 
 OpenDate:  2010 / 10 / 12 
 CloseDate:  2012 / 10 / 13 
 MVD 
 Paul 
 VIC 
 AU 
 /03/ / 87 / 19 
 A


In [89]:
# DUMMY CODE FOR TRIALS AND TESTING

# mycursor=instance.cursor(buffered=True)
# coun='customer'
# flag=0
# stmt = "show tables;"
# mycursor.execute(stmt)
# result =mycursor.fetchall()
# # print(result)
# for i in result:
# #     print(i)
#     if (i[0]==coun):
#         flag=1
#         break

# if (flag==0):
    
# else:
#     print("exists")

exists


In [ ]:
# CREATE TABLE `incubyte`.`customer` (
#   `CustomerName` VARCHAR(255) NOT NULL,
#   `CustomerID` VARCHAR(18) NOT NULL,
#   `CustomerOpenDate` DATE NOT NULL,
#   `LastConsultedDate` DATE NULL,
#   `VaccineType` CHAR(5) NULL,
#   `DoctorConsulted` CHAR(255) NULL,
#   `State` CHAR(5) NULL,
#   `Country` CHAR(5) NULL,
#   `PostCode` INT(5) NULL,
#   `DOB` DATE NULL,
#   `ActiveCustomer` CHAR(1) NULL,
#   PRIMARY KEY (`CustomerName`));
--------------------------------------------
# try:
#    conn.commit()

# except:
#    conn.rollback()